In [4]:
import pandas as pd
import numpy as np
import requests
import datetime
from bs4 import BeautifulSoup
import json
from os import listdir
import time
import os

In [5]:
# folder yang akan digunakan

# folder ini berisi file json judul artikel dari kategori market
if not os.path.exists('data/json_bisnis_market'):
    os.makedirs('data/json_bisnis_market')

# folder ini berisi file json judul artikel dari kategori finance
if not os.path.exists('data/json_bisnis_finance'):
    os.makedirs('data/json_bisnis_finance')

# folder ini berisi file json per hari untuk isi berita dari kategori finance dan market
if not os.path.exists('data/isi_berita_bisnis'):
    os.makedirs('data/isi_berita_bisnis')

## Scraping Judul

In [37]:
def scrape_page(halaman, tanggal, c):
    """Melakukan request kehalaman bisnis.com 
    pada tanggal tertentu dengan halaman tertentu
    pada kategori (c) tertentu"""
    
    URL = "https://www.bisnis.com/index/page/?c={}&d={}&per_page={}".format(c, tanggal, halaman)
    page = requests.get(URL, headers={'user-agent': 'My app'})
    return BeautifulSoup(page.content, 'html.parser')

def extract_informasi_berita(berita):
    """Mengambil informasi judul berita, link, dan waktu publish"""
    
    div = berita.find('div', class_="col-sm-8")
    a = div.select("h2 a")[0]
    return {
        "judul" : a['title'],
        "kategori": div.select(".wrapper-description a")[0]['title'],
        "publish_date" : div.select(".date")[0].text.strip(),
        "link": a['href']
    }

def scrape_tanggal(tanggal, c):
    """Scraping pada tanggal tertentu"""
    
    halaman_sekarang = 1
    beritas = []

    # Dalam satu hari bisa terdiri dari banyak berita
    # sehingga bisa terdiri dari banyak halaman
    while True:
        soup = scrape_page(halaman_sekarang, tanggal, c)
        list_berita = soup.find("ul", class_="list-news").find_all("li")
        
        # Jika list berita kosong maka sudah sampai dihalaman terakhir
        if (len(list_berita) == 1) and (list_berita[0].find("h2").text.strip() == "Tidak ada berita"):
            break
            
        for berita in list_berita:
            beritas.append(extract_informasi_berita(berita))

        halaman_sekarang += 1
        
    return beritas, halaman_sekarang - 1

def generate_date_range(start, end=None):
    """Generate tanggal dengan rentang tertentu. Parameter
    'Start' dan 'End' harus berformat d-m-Y. Misalnya 27-01-2001
    """
    
    start = datetime.datetime.strptime(start, "%d-%m-%Y")
    if end is None:
        end = datetime.datetime.today()
    else:
        end = datetime.datetime.strptime(end, "%d-%m-%Y")
    return [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

def timestamp_to_time(timestamp):
    """Convert timestamp ke dalam bentuk datetime"""
    
    dt_object = datetime.datetime.fromtimestamp(timestamp)
    return dt_object.strftime("%Y-%m-%d %H:%M:%S")

In [38]:
def scrape_bisnis_finance(start, c, folder_json, file_csv, end=None, file_ke=1, path_file="data"):
    """Scraping untuk rentang tertentu. Hasilnya akan disimpan pada folder data
    masih dalam bentuk json karena akan dilakukan penyimpanan setiap 150 hari
    """
    
    generated_date = generate_date_range(start)
    hasil = []
    status = []
    status2 = None
    
    # Cek apakah sudah pernah melakukan scraping halaman bisnis.com sebelumnya
    # jika iya ambil file terakhir sehingga hasil scraping bisa berlanjut
    # dan tidak menimpa file yang sudah ada
    try:
        status2 = pd.read_csv(f'{path_file}/{file_csv}')
        file_ke = int(status2.file_ke.dropna().max()) + 1
    except:
        pass
    
    iterate = 1
    for date in generated_date:
        tanggal_ymd = date.strftime("%Y-%m-%d")
        if len(hasil) == 0:
            tanggal_mulai = tanggal_ymd
        
        start_scraping = int(datetime.datetime.timestamp(datetime.datetime.now()))
        
        print(f"{iterate}. Mulai scraping tanggal {tanggal_ymd} at {timestamp_to_time(start_scraping)}")
        jumlah_berita = 0
        
        try:
            list_berita, jumlah_halaman = scrape_tanggal(tanggal_ymd, c)
            end_scraping = int(datetime.datetime.timestamp(datetime.datetime.now()))
            status_scrape = "berhasil"
            jumlah_berita = len(list_berita)

            hasil.append({ "tanggal" : tanggal_ymd,
                          "jumlah_berita": jumlah_berita,
                          "data" : list_berita})
            
            status.append({ "tanggal" : tanggal_ymd,
                           "status" : status_scrape,
                           "jumlah_berita": jumlah_berita,
                           "start_scrape" : start_scraping,
                           "end_scrape" : end_scraping,
                           "jumlah_halaman" : jumlah_halaman,
                           "file_ke" : file_ke})
        except Exception as e:
            print(e)
            status_scrape = "gagal"
            status.append({ "tanggal" : tanggal_ymd,
                           "status" : status_scrape,
                           "jumlah_berita": None,
                           "start_scrape" : None,
                           "end_scrape" : None,
                           "jumlah_halaman" : None,
                           "file_ke" : None})
        
        print(f"Scraping tanggal {tanggal_ymd} {status_scrape}, jumlah berita {jumlah_berita}\n")
        # setiap iterasi kelipatan 150, save datanya dan kosongkan variabel hasil
        # variabel hasil dikosongkan agar hemat memori dan ketika disimpan tidak 
        # membutuhkan waktu yang lama
        try:
            if iterate % 150 == 0:
                with open(f'{path_file}/{folder_json}/{file_ke}_{tanggal_mulai}_{tanggal_ymd}.txt', 'w', encoding='utf-8') as f:
                    json.dump(hasil, f, ensure_ascii=False)
                hasil = []
                file_ke += 1
                
                s = pd.DataFrame(status)
                if status2 is None:
                    s = pd.concat([status2, s]).sort_values('tanggal')
                s.to_csv(f"{path_file}/{file_csv}")
                
            iterate += 1
        except:
            break

    # save iterasi yang terakhir
    try:
        with open(f'{path_file}/{folder_json}/{file_ke}_{tanggal_mulai}_{tanggal_ymd}.txt', 'w', encoding='utf-8') as f:
            json.dump(hasil, f, ensure_ascii=False)

        s = pd.DataFrame(status)            
        if status2 is None:
            s = pd.concat([status2, s]).sort_values('tanggal')
        s.to_csv(f"{path_file}/{file_csv}")
    except:
        print("Terjadi error")
        
    return status, hasil

In [25]:
# c = 194 adalah kategori artikel market di bisnis.com
status, hasil = scrape_bisnis_finance("30-04-2011", c=194, folder_json="json_bisnis_market", file_csv="status_bisnis_market.csv")
status

In [ ]:
# c = 5 adalah kategori artikel financial di bisnis.com
status, hasil = scrape_bisnis_finance("01-12-2010", 5, "json_bisnis_finance", "status_bisnis_finance.csv")
status

## Satukan Hasil Scraping

### Kategori Finance

In [41]:
data = pd.read_csv("data/status_bisnis_finance.csv", index_col=0)
data

,tanggal,status,jumlah_berita,start_scrape,end_scrape,jumlah_halaman,file_ke
0,2010-12-01,berhasil,2.0,1.639150e+09,1.639150e+09,1.0,1.0
1,2010-12-02,berhasil,1.0,1.639150e+09,1.639150e+09,1.0,1.0
2,2010-12-03,berhasil,0.0,1.639150e+09,1.639150e+09,0.0,1.0
3,2010-12-04,berhasil,1.0,1.639150e+09,1.639150e+09,1.0,1.0
4,2010-12-05,berhasil,1.0,1.639150e+09,1.639150e+09,1.0,1.0
...,...,...,...,...,...,...,...
4022,2021-12-05,berhasil,12.0,1.639152e+09,1.639152e+09,1.0,27.0
4023,2021-12-06,berhasil,40.0,1.639152e+09,1.639152e+09,3.0,27.0
4024,2021-12-07,berhasil,56.0,1.639152e+09,1.639152e+09,4.0,27.0
4025,2021-12-08,berhasil,32.0,1.639152e+09,1.639152e+09,3.0,27.0


In [43]:
data.status.value_counts()

berhasil    4025
gagal          2
Name: status, dtype: int64

In [44]:
data[data.status == "gagal"]

,tanggal,status,jumlah_berita,start_scrape,end_scrape,jumlah_halaman,file_ke
1460,2014-11-30,gagal,NaN,NaN,NaN,NaN,NaN
2800,2018-08-01,gagal,NaN,NaN,NaN,NaN,NaN


In [50]:
# cek tanggal yang gagal discarping, untuk dilakukan scripng ulang
tanggal_gagal = ["2014-11-30", "2018-08-01"]

hasil = []
for tanggal in tanggal_gagal:
    list_berita, jumlah_halaman = scrape_tanggal(tanggal, 5)
    hasil.append({ "tanggal" : tanggal,
                  "jumlah_berita": len(list_berita),
                  "data" : list_berita})
    
with open('data/json_bisnis_finance/0_tanggal_gagal.txt', 'w', encoding='utf-8') as f:
    json.dump(hasil, f, ensure_ascii=False)  

In [52]:
# untuk read data json judul dan dijadikan satu dataframe

path = "data/json_bisnis_finance"
files = listdir(path)

data = None
for file_txt in files:
    try:
        with open(f"{path}/{file_txt}", "r", encoding="utf-8") as f:
            isi = json.load(f)
            f.close()
    except:
        with open(f"{path}/{file_txt}", "r") as f:
            isi = json.load(f)
            f.close()
    
    if data is None:
        data = pd.DataFrame(isi)
    else:
        data = pd.concat([data, pd.DataFrame(isi)])
        
data

,tanggal,jumlah_berita,data
0,2014-11-30,12,[{'judul': 'BRANCHLESS BANKING: Bank Bukopin A...
1,2018-08-01,16,[{'judul': 'Direktur Mandiri Terpilih Kembali ...
0,2014-08-12,18,"[{'judul': 'KINERJA ASURANSI: Premi Naik 101%,..."
1,2014-08-13,27,[{'judul': 'GERAKAN NON TUNAI: BI Puji Pemda D...
2,2014-08-14,33,[{'judul': 'ASET BUMN BERMASALAH: Menteri BUMN...
...,...,...,...
145,2014-08-07,28,[{'judul': 'BANK INDONESIA Latih Petani Bawang...
146,2014-08-08,9,[{'judul': 'Perbankan Syariah Perlu Turunkan B...
147,2014-08-09,7,[{'judul': 'TRUST FINANCE (TRUS): Pendapatan M...
148,2014-08-10,0,[]


In [65]:
# memisahkan kolom data menjadi beberapa kolom

data_full = None

for index, row in data.iterrows():
    data_satu_hari = pd.DataFrame(row['data'])
    data_satu_hari['tanggal'] = row['tanggal']
    
    if data_full is None:
        data_full = pd.DataFrame(data_satu_hari)
    else:
        data_full = pd.concat([data_full, pd.DataFrame(data_satu_hari)])

data_full.publish_date = pd.to_datetime(data_full.publish_date, format="%d %b %Y | %H:%M WIB")
data_full = data_full.sort_values("publish_date")
data_full_finance = data_full.reset_index(drop=True)

data_full_finance

,judul,kategori,publish_date,link,tanggal
0,PPN kegiatan usaha bank umum dipertegas,Perbankan,2010-12-01 03:23:00,https://finansial.bisnis.com/read/20101201/90/...,2010-12-01
1,"BI: Kredit bank sepekan tumbuh Rp10,47 triliun",Perbankan,2010-12-01 03:36:00,https://finansial.bisnis.com/read/20101201/90/...,2010-12-01
2,Parlemen incar penetapan dewan komisioner OJK,Perbankan,2010-12-02 14:03:00,https://finansial.bisnis.com/read/20101202/90/...,2010-12-02
3,Mandiri tawarkan klausula arbitrase di perjanj...,Perbankan,2010-12-04 04:50:00,https://finansial.bisnis.com/read/20101204/90/...,2010-12-04
4,Ada anomali pada kredit perbankan,Perbankan,2010-12-05 13:32:00,https://finansial.bisnis.com/read/20101205/90/...,2010-12-05
...,...,...,...,...,...
55839,BCA Pastikan Kantor Cabang di Lumajang Tetap B...,Perbankan,2021-12-09 18:50:00,https://finansial.bisnis.com/read/20211209/90/...,2021-12-09
55840,Fintech Wealth-Tech Harus Ikut Jaga Gairah Inv...,Fintech,2021-12-09 19:10:00,https://finansial.bisnis.com/read/20211209/563...,2021-12-09
55841,"Bank Mandiri (BMRI) Suntik Pembiayaan Rp2,45 T...",Perbankan,2021-12-09 19:22:00,https://finansial.bisnis.com/read/20211209/90/...,2021-12-09
55842,"Tak Bagi Dividen, Bank of India Indonesia (BSW...",Perbankan,2021-12-09 19:31:00,https://finansial.bisnis.com/read/20211209/90/...,2021-12-09


### Kategori Market

In [66]:
data = pd.read_csv("data/status_bisnis_market.csv", index_col=0)
data

,tanggal,status,jumlah_berita,start_scrape,end_scrape,jumlah_halaman,file_ke
0,2011-04-30,berhasil,0.0,1.639137e+09,1.639137e+09,0.0,2.0
1,2011-05-01,berhasil,11.0,1.639137e+09,1.639137e+09,1.0,2.0
2,2011-05-02,berhasil,16.0,1.639137e+09,1.639137e+09,2.0,2.0
3,2011-05-03,berhasil,24.0,1.639137e+09,1.639137e+09,2.0,2.0
4,2011-05-04,berhasil,21.0,1.639137e+09,1.639137e+09,2.0,2.0
...,...,...,...,...,...,...,...
3872,2021-12-05,berhasil,13.0,1.639141e+09,1.639141e+09,1.0,27.0
3873,2021-12-06,berhasil,77.0,1.639141e+09,1.639141e+09,6.0,27.0
3874,2021-12-07,berhasil,67.0,1.639141e+09,1.639141e+09,5.0,27.0
3875,2021-12-08,berhasil,69.0,1.639141e+09,1.639141e+09,5.0,27.0


In [67]:
data.status.value_counts()

berhasil    3131
gagal        746
Name: status, dtype: int64

In [68]:
data[data.status == "gagal"]

,tanggal,status,jumlah_berita,start_scrape,end_scrape,jumlah_halaman,file_ke
112,2011-08-20,gagal,NaN,NaN,NaN,NaN,NaN
650,2013-02-08,gagal,NaN,NaN,NaN,NaN,NaN
671,2013-03-01,gagal,NaN,NaN,NaN,NaN,NaN
674,2013-03-04,gagal,NaN,NaN,NaN,NaN,NaN
675,2013-03-05,gagal,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2776,2018-12-05,gagal,NaN,NaN,NaN,NaN,NaN
2777,2018-12-06,gagal,NaN,NaN,NaN,NaN,NaN
2781,2018-12-10,gagal,NaN,NaN,NaN,NaN,NaN
2782,2018-12-11,gagal,NaN,NaN,NaN,NaN,NaN


In [ ]:
# scripng ulang data yang gagal
tanggal_gagal = data[data.status == "gagal"].tanggal.to_list()

hasil = []
for tanggal in tanggal_gagal:
    print("Scrape tanggal", tanggal, "\n")
    list_berita, jumlah_halaman = scrape_tanggal(tanggal, 5)
    hasil.append({ "tanggal" : tanggal,
                  "jumlah_berita": len(list_berita),
                  "data" : list_berita})
    
with open('data/json_bisnis_market/0_tanggal_gagal.txt', 'w', encoding='utf-8') as f:
    json.dump(hasil, f, ensure_ascii=False)  

In [72]:
# satukan data json judul dari kategori market
path = "data/json_bisnis_market"
files = listdir(path)

data = None
for file_txt in files:
    try:
        with open(f"{path}/{file_txt}", "r", encoding="utf-8") as f:
            isi = json.load(f)
            f.close()
    except:
        with open(f"{path}/{file_txt}", "r") as f:
            isi = json.load(f)
            f.close()
    
    if data is None:
        data = pd.DataFrame(isi)
    else:
        data = pd.concat([data, pd.DataFrame(isi)])
        
data

,tanggal,jumlah_berita,data
0,2011-08-20,2,[{'judul': 'Pemerintah klaim 10% debitur KUR '...
1,2013-02-08,2,[{'judul': 'BNI Terbitkan 60.000 Unit Kartu Co...
2,2013-03-01,5,[{'judul': 'LIMIT KARTU KREDIT: Akan Diturunka...
3,2013-03-04,2,[{'judul': 'Perusahaan Akan Dipaksa Ikut Progr...
4,2013-03-05,11,[{'judul': 'PEMILIHAN GUBERNUR BI: 4 Fraksi DP...
...,...,...,...
138,2014-08-07,58,[{'judul': 'OBLIGASI PEMERINTAH (SUN): Ekonom ...
139,2014-08-08,69,[{'judul': 'MALAYSIAN AIRLINES Berpotensi Deli...
140,2014-08-09,6,[{'judul': 'KALBE FARMA (KLBF): Cetak Pendapat...
141,2014-08-10,5,[{'judul': 'GARUDA INDONESIA (GIAA) Dorong GMF...


In [73]:
# memisahkan kolom data menjadi kolom-kolom terpisah
data_full = None

for index, row in data.iterrows():
    data_satu_hari = pd.DataFrame(row['data'])
    data_satu_hari['tanggal'] = row['tanggal']
    
    if data_full is None:
        data_full = pd.DataFrame(data_satu_hari)
    else:
        data_full = pd.concat([data_full, pd.DataFrame(data_satu_hari)])

data_full.publish_date = pd.to_datetime(data_full.publish_date, format="%d %b %Y | %H:%M WIB")
data_full = data_full.sort_values("publish_date")
data_full_market = data_full.reset_index(drop=True)

data_full_market

,judul,kategori,publish_date,link,tanggal
0,Krisis utang Eropa tekan harga saham,Saham & Sekuritas,2010-12-01 00:48:00,https://market.bisnis.com/read/20101201/190/25...,2010-12-01
1,Nusantara Infrastructure bidik pendapatan Rp45...,Korporasi,2010-12-01 01:11:00,https://market.bisnis.com/read/20101201/192/18...,2010-12-01
2,Jasindo kembangkan bisnis syariah,Korporasi,2010-12-01 01:32:00,https://market.bisnis.com/read/20101201/192/15...,2010-12-01
3,Harga minyak terkikis spekulasi dana talangan ...,Emas,2010-12-01 01:36:00,https://market.bisnis.com/read/20101201/235/13...,2010-12-01
4,Harga emas naik dapat berkah dari krisis Eropa,Emas,2010-12-01 01:39:00,https://market.bisnis.com/read/20101201/235/17...,2010-12-01
...,...,...,...,...,...
125992,"Lakukan Restrukturisasi dan Terdampak PPKM, In...",Korporasi,2021-12-09 20:20:00,https://market.bisnis.com/read/20211209/192/14...,2021-12-09
125993,"Resmi PKPU, Garuda Indonesia (GIAA) Pastikan O...",Korporasi,2021-12-09 20:37:00,https://market.bisnis.com/read/20211209/192/14...,2021-12-09
125994,"Daftar Saham-Saham Syariah JII Favorit, Simak ...",Rekomendasi,2021-12-09 20:48:00,https://market.bisnis.com/read/20211209/189/14...,2021-12-09
125995,"Besok IHSG Diprediksi Menguat, Cek Saham CTRA,...",Rekomendasi,2021-12-09 21:09:00,https://market.bisnis.com/read/20211209/189/14...,2021-12-09


### Gabungkan Kedua Kategori

In [76]:
# menggabungkan antara data kategori market dan finance menjadi satu file
data_full = pd.concat([data_full_finance, data_full_market]).sort_values("publish_date").reset_index(drop=True)
data_full

,judul,kategori,publish_date,link,tanggal
0,Krisis utang Eropa tekan harga saham,Saham & Sekuritas,2010-12-01 00:48:00,https://market.bisnis.com/read/20101201/190/25...,2010-12-01
1,Nusantara Infrastructure bidik pendapatan Rp45...,Korporasi,2010-12-01 01:11:00,https://market.bisnis.com/read/20101201/192/18...,2010-12-01
2,Jasindo kembangkan bisnis syariah,Korporasi,2010-12-01 01:32:00,https://market.bisnis.com/read/20101201/192/15...,2010-12-01
3,Harga minyak terkikis spekulasi dana talangan ...,Emas,2010-12-01 01:36:00,https://market.bisnis.com/read/20101201/235/13...,2010-12-01
4,Harga emas naik dapat berkah dari krisis Eropa,Emas,2010-12-01 01:39:00,https://market.bisnis.com/read/20101201/235/17...,2010-12-01
...,...,...,...,...,...
181836,"Resmi PKPU, Garuda Indonesia (GIAA) Pastikan O...",Korporasi,2021-12-09 20:37:00,https://market.bisnis.com/read/20211209/192/14...,2021-12-09
181837,"Daftar Saham-Saham Syariah JII Favorit, Simak ...",Rekomendasi,2021-12-09 20:48:00,https://market.bisnis.com/read/20211209/189/14...,2021-12-09
181838,"Besok IHSG Diprediksi Menguat, Cek Saham CTRA,...",Rekomendasi,2021-12-09 21:09:00,https://market.bisnis.com/read/20211209/189/14...,2021-12-09
181839,"Pelita Samudera (PSSI) Eksekusi Buyback 109,67...",Korporasi,2021-12-09 21:32:00,https://market.bisnis.com/read/20211209/192/14...,2021-12-09


In [78]:
data_full.to_csv("data/link_berita_bisnis.csv", index=False)

## Scraping Satu Halaman

Melakukan scriping setiap halaman, setiap link yang telah dikumpulkan akan dikunjungi satu per satu

In [6]:
data_judul = pd.read_csv("data/link_berita_bisnis.csv")
data_judul

,judul,kategori,publish_date,link,tanggal
0,Krisis utang Eropa tekan harga saham,Saham & Sekuritas,2010-12-01 00:48:00,https://market.bisnis.com/read/20101201/190/25...,2010-12-01
1,Nusantara Infrastructure bidik pendapatan Rp45...,Korporasi,2010-12-01 01:11:00,https://market.bisnis.com/read/20101201/192/18...,2010-12-01
2,Jasindo kembangkan bisnis syariah,Korporasi,2010-12-01 01:32:00,https://market.bisnis.com/read/20101201/192/15...,2010-12-01
3,Harga minyak terkikis spekulasi dana talangan ...,Emas,2010-12-01 01:36:00,https://market.bisnis.com/read/20101201/235/13...,2010-12-01
4,Harga emas naik dapat berkah dari krisis Eropa,Emas,2010-12-01 01:39:00,https://market.bisnis.com/read/20101201/235/17...,2010-12-01
...,...,...,...,...,...
181836,"Resmi PKPU, Garuda Indonesia (GIAA) Pastikan O...",Korporasi,2021-12-09 20:37:00,https://market.bisnis.com/read/20211209/192/14...,2021-12-09
181837,"Daftar Saham-Saham Syariah JII Favorit, Simak ...",Rekomendasi,2021-12-09 20:48:00,https://market.bisnis.com/read/20211209/189/14...,2021-12-09
181838,"Besok IHSG Diprediksi Menguat, Cek Saham CTRA,...",Rekomendasi,2021-12-09 21:09:00,https://market.bisnis.com/read/20211209/189/14...,2021-12-09
181839,"Pelita Samudera (PSSI) Eksekusi Buyback 109,67...",Korporasi,2021-12-09 21:32:00,https://market.bisnis.com/read/20211209/192/14...,2021-12-09


untuk memantau hasil scraping maka akan dibuat file status_isi_bisnis yang berisi index dan status berhasil scraping atau tidak misalnya scaraping tiba tiba gagal maka cukup melihat data di file tersebut untuk mengetahui sudah sampai mana proses scrapingnya sehingga tidak mulai dari awal lagi

In [59]:
iterasi = 0
status = []
last_index = 0

try:
    data_status = pd.read_csv("data/status_isi_bisnis.csv", index_col=0)
    status = data_status.to_dict('records')
    last_index = data_status.index[-1]
except:
    pass

for tanggal, df, in data_judul.iloc[last_index +  1:, :].groupby("tanggal"):
    data_per_tanggal = []
    print(f"Scraping tanggal {tanggal}")
    
    for index, row in df.iterrows():
        tmp = {'index' : index, 'judul' : row['judul'], 'link' : row['link'], 'tanggal' : tanggal}
        
        try:
            page = requests.get(tmp['link'], headers={'user-agent': 'My app'})
            page = BeautifulSoup(page.content, 'html.parser')

            article_body = page.select(".description .sticky-wrapper .col-sm-10")[0]
            paragraphs = article_body.find_all("p")
            isi_artikel = " ".join([paragraph.text.strip() for paragraph in paragraphs[:-2]])    
            tmp['is_sukses'] = True
            tmp['isi'] = isi_artikel
            status.append({'index': index, "is_sukses" : True})
            
        except:
            tmp['is_sukses'] = False
            tmp['isi'] = None
            status.append({'index': index, "is_sukses" : False})
            time.sleep(3)
        
        data_per_tanggal.append(tmp)
        iterasi += 1
        
        # setiap 200 iterasi, istirahat 15 detik.
        if iterasi % 200 == 0:
            time.sleep(15)
    
    pd.DataFrame(status).to_csv("data/status_isi_bisnis.csv")
    try:
        with open(f'data/isi_berita_bisnis/{tanggal}.txt', 'w', encoding='utf-8') as f:
            json.dump(data_per_tanggal, f, ensure_ascii=False)
    except:
        print("Terjadi error")
        break

Scraping tanggal 2021-06-04
Scraping tanggal 2021-06-05
Scraping tanggal 2021-06-06
Scraping tanggal 2021-06-07
Scraping tanggal 2021-06-08
Scraping tanggal 2021-06-09
Scraping tanggal 2021-06-10
Scraping tanggal 2021-06-11
Scraping tanggal 2021-06-12
Scraping tanggal 2021-06-13
Scraping tanggal 2021-06-14
Scraping tanggal 2021-06-15
Scraping tanggal 2021-06-16
Scraping tanggal 2021-06-17
Scraping tanggal 2021-06-18
Scraping tanggal 2021-06-19
Scraping tanggal 2021-06-20
Scraping tanggal 2021-06-21
Scraping tanggal 2021-06-22
Scraping tanggal 2021-06-23
Scraping tanggal 2021-06-24
Scraping tanggal 2021-06-25
Scraping tanggal 2021-06-26
Scraping tanggal 2021-06-27
Scraping tanggal 2021-06-28
Scraping tanggal 2021-06-29
Scraping tanggal 2021-06-30
Scraping tanggal 2021-07-01
Scraping tanggal 2021-07-02
Scraping tanggal 2021-07-03
Scraping tanggal 2021-07-04
Scraping tanggal 2021-07-05
Scraping tanggal 2021-07-06
Scraping tanggal 2021-07-07
Scraping tanggal 2021-07-08
Scraping tanggal 202

## Scraping yang Gagal

In [8]:
status_isi = pd.read_csv('data/status_isi_bisnis.csv', index_col=0)
unsuccess = status_isi[status_isi.is_sukses == False].index
unsuccess

Int64Index([   769,   1350,   1761,   1789,   1874,   2044,   2696,   4309,
              4432,   5001,
            ...
            181735, 181751, 181754, 181779, 181803, 181806, 181810, 181822,
            181827, 181834],
           dtype='int64', length=3959)

In [9]:
len(unsuccess)

3959

In [12]:
data_judul.iloc[unsuccess, :]

,judul,kategori,publish_date,link,tanggal
769,Bank OCBC NISP peroleh peringkat idAA+,Perbankan,2011-01-13 09:06:00,https://finansial.bisnis.com/read/20110113/90/...,2011-01-13
1350,Fitch: Peringkat utang Lippo B+,Saham & Sekuritas,2011-02-10 08:01:00,https://market.bisnis.com/read/20110210/190/22...,2011-02-10
1761,Peringkat obligasi Indofood naik jadi idAA+,Obligasi & Reksadana,2011-05-04 10:40:00,https://market.bisnis.com/read/20110504/92/313...,2011-05-04
1789,Pefindo beri idA+ ke Surya Citra Televisi,Obligasi & Reksadana,2011-05-05 10:22:00,https://market.bisnis.com/read/20110505/92/311...,2011-05-05
1874,Obligasi global Pertamina mendapat peringkat B...,Obligasi & Reksadana,2011-05-10 11:40:00,https://market.bisnis.com/read/20110510/92/306...,2011-05-10
...,...,...,...,...,...
181806,Boy Thohir Antarkan Produsen Batu Bara Kokas A...,Bursa & Saham,2021-12-09 16:00:00,https://market.bisnis.com/read/20211209/7/1475...,2021-12-09
181810,Prospek Aksi Akuisisi Medco MEDC di Tengah Tre...,Korporasi,2021-12-09 16:34:00,https://market.bisnis.com/read/20211209/192/14...,2021-12-09
181822,"Reli Harga Acuan Terhenti, Pesta Saham Batu Ba...",Korporasi,2021-12-09 18:30:00,https://market.bisnis.com/read/20211209/192/14...,2021-12-09
181827,Restrukturisasi Utang Pan Brothers PBRX Dikabu...,Korporasi,2021-12-09 19:09:00,https://market.bisnis.com/read/20211209/192/14...,2021-12-09


ternyata yang gagal itu yang halamannya memang tidak ada atau konten premium sehingga tidak akan dilakukan scraping ulang